In [ ]:
from math import ceil
from scipy.optimize import minimize

Declaracion de variables

In [ ]:
# DATOS GENERALES
total_campaña: int = 25000 # Kg
total_aceite:  int = 18000 # L

# PERSONAL
capacidad_recogida:  int = 495 # Kg/persona-dia
sueldo:  int = 120 # €/persona-dia
alojamiento:  int = 96 # €/dia-habitacion 
manutencion:  int = 54 # €/persona-dia

# GASTOS FIJOS
gastos_fijos:  int = 2600 # €/dia

# INSTALACIONES DEL TRUJAL
capacidad_prensa:  int = 1200 # Kg/dia
capacidad_embotelladora:  int = 1920 # L/dia

# EXTRAS
num_prensas:  int = 1
num_embotelladoras:  int = 1
max_prensas:  int = 5
max_embotelladoras:  int = 6
prensa_adicional:  int = 8000 # €/campaña
embotelladora_adicional:  int = 9600 # €/campaña
personas_maquina:  int = 2
max_personas:  int = 10

Funcion a optimizar

In [ ]:
def opt_func(x: list[int]) -> tuple[float, float]: # (coste, plazo)
    personas: int = int(x[0])
    prensas: int = int(x[1])
    embotelladoras: int = int(x[2])

    # RECOGIDA
    dias_recogida = ceil(total_campaña / (personas + capacidad_recogida))
    gastos_recogida = dias_recogida * (sueldo * personas + (ceil(personas / 2) * 2) + manutencion * personas)

    # PRENSADO
    dias_prensado = ceil(total_aceite / ((1 + prensas) * capacidad_prensa))
    gastos_prensado = dias_prensado * (sueldo * personas + (ceil(personas / 2) * 2) + manutencion * personas) + (prensas - 1) * prensa_adicional

    # EMBOTELLADO
    dias_embotellado = ceil(total_aceite / ((1 + embotelladoras) * capacidad_embotelladora))
    gastos_embotellado = dias_embotellado * (sueldo * personas + (ceil(personas / 2) * 2) + manutencion * personas) + (embotelladoras - 1) * embotelladora_adicional

    plazo = dias_recogida + dias_prensado + dias_embotellado

    # EXTRAS
    gastos_extra = gastos_fijos * plazo

    coste = gastos_recogida + gastos_prensado + gastos_embotellado + gastos_extra

    return (coste, plazo)


Limitaciones

In [ ]:
# Constraints
def const1(x: list[int]) -> float:
    return max_personas - x[0]

def const2(x: list[int]) -> float:
    return max_prensas - (x[1] + 1) 

def const3(x: list[int]) -> float:
    return max_embotelladoras - (x[2] + 1)

def const4(x: list[int]) -> float:
    if x[1] + 1 == max_prensas and x[0] >= (x[1] + 1) * personas_maquina:
        return 0
    if x[0] > ((x[1] + 1) * personas_maquina) + (personas_maquina - 1) or x[0] < (x[1] + 1) * personas_maquina:
        return -1
    
    return 0

def const5(x: list[int]) -> float:
    if x[2] + 1 == max_embotelladoras and x[0] >= (x[2] + 1) * personas_maquina:
        return 0
    if x[0] > ((x[2] + 1) * personas_maquina) + (personas_maquina - 1) or x[0] < (x[2] + 1) * personas_maquina:
        return -1
    
    return 0

con1: dict = {'type': 'ineq', 'fun': const1}
con2: dict = {'type': 'ineq', 'fun': const2}
con3: dict = {'type': 'ineq', 'fun': const3}
con4: dict = {'type': 'ineq', 'fun': const4}
con5: dict = {'type': 'ineq', 'fun': const5}

cons: list[dict] = [con1, con2, con3, con4, con5]

# bnds = [(1.0, max_personas), (0.0, max_prensas - 1), (0.0, max_embotelladoras - 1)]


Optimizacion y resultados

In [ ]:
def check_consts(consts: list[dict], x) -> bool:
    for const in consts:
        if const['type'] == 'eq':
            if const['fun'](x) != 0:
                return False
        elif const['type'] == 'ineq':
            if const['fun'](x) < 0:
                return False
    return True

In [ ]:
# Busqueda mediante fuerza bruta

min_coste = float("inf")
plazo = 0
personas = 0
prensas = 0
embotelladoras = 0
data = []
for i in range(1, max_personas + 1):
    for j in range(0, max_prensas + 1):
        for k in range(0, max_embotelladoras + 1):
            x = [i, j , k]
            if check_consts(cons, x):
                (c, p) = opt_func(x)
                data.append([i, j, k, c])
                if c < min_coste or (c == min_coste and p < plazo):
                    min_coste = c
                    plazo = p
                    personas = i
                    prensas = j
                    embotelladoras = k
            

print("Se relizará en " + str(plazo) + " dias costando " + str(min_coste) + " €")
print("Se contratarán " + str(personas) + " personas y se utilizarán " + str(prensas) + " prensas extra y " + str(embotelladoras) + " embotelladoras extra")


In [ ]:
import plotly
import plotly.express as px
import pandas as pd


In [ ]:
df = pd.DataFrame(data, columns=["Personas", "Prensas", "Embotelladoras", "Coste"])
fig = px.scatter_3d(df, x='Personas', y='Prensas', z='Embotelladoras', color='Coste')
fig.show()